# Lab 5: Using LangChain with IBM GenAI Library

## 1. Intro to LangChain

[LangChain](https://docs.langchain.com/docs/) is an open-source development framework desinged to simplify the creation of applications using large language models (LLMs).

The core idea of the library is that we can "chain" together different components to create more advanced use cases around LLMs. Here are the main components for the LangChain

- Model: interact with various LLMs
- Prompts: text that is sent to the LLMs
- Chains: allow to combine different LLM calls and actions automatically
- Embeddings and Vector Stores: break large data into chunks and store those to be queried when relevatnt
- Agents: enbale the LLMs to dynamically decide which tools to use in order to best respond to a given query

In short, **Langchain is a framework that can orchestrate a series of prompts to achieve a desired outcomes.**


## 2. How to connect LangChain to WatsonX.ai

In [18]:
import os
from dotenv import load_dotenv
try:
    from langchain import PromptTemplate
    from langchain.chains import LLMChain, SimpleSequentialChain
    from langchain.document_loaders import PyPDFLoader
    from langchain.indexes import VectorstoreIndexCreator #vectorize db index with chromadb
    from langchain.embeddings import HuggingFaceEmbeddings #for using HugginFace embedding models
    from langchain.text_splitter import CharacterTextSplitter #text splitter
except ImportError:
    raise ImportError("Could not import langchain: Please install ibm-generative-ai[langchain] extension.")

from genai.extensions.langchain import LangChainInterface
from genai.model import Credentials
from genai.schemas import GenerateParams, ModelType

In [2]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
if api_key is None or api_url is None:
    print("Either api_key or api_url is None. Please make sure your credentials are correct.")
creds = Credentials(api_key, api_url)

In [3]:
##initializing WatsonX model
params = GenerateParams(
    decoding_method="sample",
    max_new_tokens=100,
    min_new_tokens=1,
    stream=False,
    temperature=0.5,
    top_k=50,
    top_p=1,
).dict()  # Langchain uses dictionaries to pass kwargs - parameters for the model
llm_model = LangChainInterface(model=ModelType.FLAN_UL2, credentials=creds, params=params)

In [4]:
##predict with the model
text = "Where is the capital of South Korea"
llm_model(text)

'seoul'

## 3. Prompt Templates & Chains

In the previous example, the user input is sent directly to the LLM. However, when using an LLM in an application, you will usually need to reuse the same prompt across multiple scenarios

- Accepting user input and contruct a prompt
- Generating mutiple prompts from an collection of data points in a dataset 

In [5]:
# Define the prompt templates
prompt = PromptTemplate(
  input_variables=["country"],
  template= "where is the capital of {country}?",
)

# Chaining 
chain = LLMChain(llm=llm_model, prompt=prompt)

# Getting predictions
countries = ["USA", "England", "Japan", "Saudi Arabia"]
for country in countries:
    response = chain.run(country)
    print(prompt.format(country=country) + " = " + response)

where is the capital of USA? = washington
where is the capital of England? = london
where is the capital of Japan? = tokyo
where is the capital of Saudi Arabia? = jeddah


## 4. Simple sequential chains
The utility of LangChain becomes apparent as we chain outputs of one model as input to another model. Here's a simple example where one generates a question which the other model answers.

LangChain determines a model's output based on its response.  In our examples, the first model creates a response to the end prompt of "Question:" which LangChain maps as an input variable called "question" which it passes to the 2nd model.

In [6]:
## Create two sequential prompts 
pt1 = PromptTemplate(input_variables=["topic"], template="Generate a random question about {topic}: Question: ")
pt2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

In [7]:
flan = LangChainInterface(model=ModelType.FLAN_UL2, credentials=creds, params=params)
model = LangChainInterface(model=ModelType.FLAN_UL2, credentials=creds)

In [8]:
prompt_to_flan = LLMChain(llm=flan, prompt=pt1)
flan_to_model = LLMChain(llm=model, prompt=pt2)
qa = SimpleSequentialChain(chains=[prompt_to_flan, flan_to_model], verbose=True)

In [9]:
qa.run("an animal")



> Entering new  chain...
Who is the largest mammal?
blue whale

> Finished chain.


'blue whale'

## 5. Easy Loading of Documents Using Lang Chain
LangChain makes it easy to extract passages from documents so that you can answering questions based on your document's content.

In [20]:
pdf='what is generative ai.pdf'
loaders = [PyPDFLoader(pdf)]

In [14]:
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loaders)

In [15]:
###initializing watsonx flan_ul2 model
params = GenerateParams(
    decoding_method="sample",
    max_new_tokens=300,
    min_new_tokens=50,
    stream=False,
    temperature=0.2,
    top_k=100,
    top_p=1,
)

model = LangChainInterface(model=ModelType.FLAN_UL2, credentials=creds, params=params)

In [16]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=model, 
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")

In [17]:
##answering based on the documents 
chain.run("what is Machine Learning?")

'Machine learning is a type of artificial intelligence. Through machine learning, practitioners develop artificial intelligence through models that can “learn” from data patterns without human direction. The unmanageably huge volume and complexity of data (unmanageable by humans, anyway) that is now being generated has increased the potential of machine learning, as well as the need for it.'